from opentelemetry import trace
from opentelemetry.exporter.otlp.proto.grpc.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import BatchSpanProcessor

# Resource can be required for some backends, e.g. Jaeger
# If resource wouldn't be set - traces wouldn't appears in Jaeger
resource = Resource(attributes={
    "service.name": "service"
})

trace.set_tracer_provider(TracerProvider(resource=resource))
tracer = trace.get_tracer(__name__)

otlp_exporter = OTLPSpanExporter(endpoint="http://localhost:4317", insecure=True)

span_processor = BatchSpanProcessor(otlp_exporter)

trace.get_tracer_provider().add_span_processor(span_processor)

with tracer.start_as_current_span("foo"):
    print("Hello world!")

# Working with Spans

In [ ]:
# %load_ext autoreload
# %autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [ ]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from examples.expositional.end2end_apps.custom_app.custom_retriever import CustomRetriever
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [ ]:
# from trulens_eval.tru import Tru
# from trulens_eval import Tru
# Tru()
# Tru().reset_database()
# Tru().start_dashboard(_dev=repo, force=True)

In [ ]:
# Create custom app:
custom_app = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
tru_custom_app = TruCustomApp(
    custom_app,
    app_id="customapp",
)

# custom_agent = custom_app.agents[0]
# tru_custom_agent = TruCustomApp(
#      custom_agent,
#      app_id="customapp_agent"
# )

In [ ]:
instruments.Instrument().print_instrumentation()

In [ ]:
tru_custom_app.print_instrumented()

In [ ]:
# tru_custom_agent.print_instrumented()

In [ ]:
with tru_custom_app as recorder:
    custom_app.respond_to_query("testing")

rec = recorder.get()

In [ ]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame(
    [(
        s.trace_id & 0xff,
        s.name,
        type(s).__name__,
        s.span_type,
        s.span_id & 0xff,
        s.parent_span_id & 0xff if s.parent_span_id else -1,
        s.attributes
    ) for s in spans],
    columns=[
        "trace_id",
        "name",
        "type",
        "span_type",
        "span_id",
        "parent_span_id",
        "attributes"
    ],
)

In [ ]:
for span in spans:
    # pprint(span)
    # pprint(span.model_dump())
    print()

In [ ]:
frozen = spans[1].freeze()

In [ ]:
from trulens_eval.trace.exporter import TraceDB, DBSpan
from trulens_eval.trace.span import Span

In [ ]:
tempdb = TraceDB(url="sqlite:///:memory:")
with tempdb:
    for span in spans:
        tempdb.add_span(span)

In [ ]:
with tempdb as s:
    print(type(s), s)
    for span in s.query(DBSpan):#.filter(DBSpan.attributes[("trulens_eval@inputs", "data")].as_string() == "testing"):
        # pprint(span.attributes)
        pprint([span.span_id, span.trace_id])
        s = span.thaw()
        print(s)
        # pprint(span.parent_span_id)

In [ ]:
int.from_bytes(int(42).to_bytes(32))

In [ ]:
from opentelemetry.trace import status as trace_status

In [ ]:
trace_status.StatusCode["OK"]